In [ ]:
import sys
import time
import numpy as np
import pandas as pd
import itertools
import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn.covariance import GraphicalLasso as sk_GL
from sklearn.covariance import empirical_covariance
from sklearn import set_config
set_config(print_changed_only=False)

from gglasso.solver.single_admm_solver import ADMM_SGL
from gglasso.solver.single_admm_solver import block_SGL
from gglasso.helper.data_generation import time_varying_power_network, group_power_network, sample_covariance_matrix
from gglasso.helper.model_selection import single_grid_search
from benchmarks import models_to_dict, sklearn_time_benchmark, admm_time_benchmark, model_solution, benchmark_parameters
from benchmarks import network_generation, hamming_distance, sparsity_benchmark, time_benchmark, dict_to_dataframe, dict_shape

from regain.covariance import GraphicalLasso as rg_GL

In [ ]:
S_dict=dict()
X_dict=dict()
Theta_dict=dict()

p_list=[300, 500, 1000]
N_list=[600, 1000, 2000]

for p, N in zip(p_list, N_list):
    S, X, Theta = network_generation(p, N, K=1, M=2)
    
    S_dict[p, N] = S
    X_dict[p, N] = X
    Theta_dict[p, N] = Theta

print("\n Shape of S_i:", dict_shape(S_dict))
print("\n Shape of X_i:", dict_shape(X_dict))
print("\n Shape of Theta_i:", dict_shape(Theta_dict))

In [ ]:
sk_params, rg_params, admm_params = benchmark_parameters()

In [ ]:
time_dict = dict()
accuracy_dict = dict()
Z_dict = dict()

for X, S in zip(list(X_dict.values()), list(S_dict.values())):
    times, accs, precs = time_benchmark(X=X, S=S, Z_model="sklearn", lambda1=0.1, n_iter=5,
                                        sk_params=sk_params, rg_params=rg_params, admm_params=admm_params)
    
    time_dict.update(times)
    accuracy_dict.update(accs)
    Z_dict.update(precs)

In [ ]:
sparsity = sparsity_benchmark(Theta_dict=Theta_dict, Z_dict=Z_dict, t_rounding=1e-4)

In [ ]:
df = dict_to_dataframe(times=time_dict, acc_dict=accuracy_dict, spars_dict=sparsity)
df.head()

In [ ]:
fig = px.scatter(df, x="time", y="accuracy", text = "name", color= "method", 
              log_y = True, facet_col='p', facet_col_wrap=3,
              labels={
                     "time": "Time, s",
                     "accuracy": "Log_distance",
                     "method": "method"
                 },
              template = "plotly_white",
              title="Log-distance between Z and Z' with respect to ADMM convergence rates")
fig.update_traces(mode='markers+lines')
fig.show()

In [ ]:
fig = px.scatter(df, x="method", y="hamming", text = "name", color= "method", 
              log_y = True, facet_col='p', facet_col_wrap=2,
              labels={
                     "time": "Time, s",
                     "accuracy": "Log_distance",
                     "method": "method"
                 },
              template = "plotly_white",
              title="Log-distance between Z and Z' with respect to ADMM convergence rates")
fig.update_traces(mode='markers+lines')
fig.show()